## Steps til at køre scriptet:
1. Tag en kopi af denne notebook INDEN ændringer laves
2. Skriv dit navn (martin/pratt/charlotte) i en string med små bogstaver i variablen nedenfor

In [1]:
#eksempel:
name = "charlotte" #skal enten være: martin/pratt/charlotte
run = "run2_valid_"

In [2]:
from __future__ import print_function
#%matplotlib inline
import argparse
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import data_utils.data_utils_celeba_pytorch5 as data_utils
import data_utils.network_tuning23 as network_tuning
from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
import pickle
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout2d, MaxPool2d, BatchNorm2d
from torch.nn.functional import relu, elu, relu6, sigmoid, softmax

In [3]:
IMAGE_SHAPE = [73,60,3]
# Paths to data
# Root directory for dataset
if name=="charlotte":
    dataroot = '..\\AlgorithmicFairness'
    TRAIN_PATH =  dataroot + "\\Data\\train.csv" 
    VALID_PATH = dataroot + "\\Data\\valid.csv" 
    TEST_PATH = dataroot + "\\Data\\test.csv" 
    IMAGE_PATHS = "C:\\Users\\cfthe\\OneDrive\\DTU\\Kandidat\\Deep\\celebA_resize3\\"

else:
    dataroot = '../AlgorithmicFairness'
    TRAIN_PATH =  dataroot + "/Data/train.csv" 
    VALID_PATH = dataroot + "/Data/valid.csv" 
    TEST_PATH = dataroot + "/Data/test.csv" 
    IMAGE_PATHS = img_path
TARGET_COL = 'Smiling'
NUM_CLASSES = 2

# train holds both X (input) and t (target/truth)
data_train = data_utils.CelebADataset(TRAIN_PATH,IMAGE_PATHS,IMAGE_SHAPE,TARGET_COL)
data_valid = data_utils.CelebADataset(VALID_PATH,IMAGE_PATHS,IMAGE_SHAPE,TARGET_COL)

#tuning the network round 1
df = pd.DataFrame(index=list(range(1,7)), columns=['layers','activations','conv_out_channels','kernel_size','conv_stride','maxpool'\
                                                    ,'dropout','batchnorm','optimizer','learning_rate','weight_decay','batch_size','num_epochs'\
                                                    ,'net','train_loss','train_accs','valid_train_loss','valid_train_accs','valid_loss','valid_accs'])

df['layers'] = 1

#adjust activation function
for i in range(1,4):
    df.at[i,'activations'] = [relu, relu, relu, relu]
for i in range(4,7):
    df.at[i,'activations'] = [torch.tanh, torch.tanh, torch.tanh, torch.tanh]


IMAGE_SHAPE = [73,60,3]
df['conv_out_channels'] = 32
df['kernel_size'] = 5
df['conv_stride'] = 1
df['maxpool'] = 1
df['dropout'] = 0.0
df['batchnorm'] = True
df['optimizer']='Adam'
df['learning_rate']=0.001
df['weight_decay']=0.01
df['batch_size']=128
df['num_epochs']=5

#adjust depth
#df.at[1:4,'layers'] = 2
#df.at[7:11,'layers'] = 2

#adjust learning rate
df.at[2,'learning_rate']=0.005
df.at[5,'learning_rate']=0.005
df.at[3,'learning_rate']=0.01
df.at[6,'learning_rate']=0.01

#adjust channels
#df.at[1,'conv_out_channels'] = 32 
#df.at[2,'conv_out_channels'] = 64
#df.at[3,'conv_out_channels'] = 32
#df.at[4,'conv_out_channels'] = 128
#df.at[5,'conv_out_channels'] = 32
#df.at[6,'conv_out_channels'] = 128 
#df.at[7,'conv_out_channels'] = 32
#df.at[8,'conv_out_channels'] = 64
#df.at[9,'conv_out_channels'] = 32
#df.at[10,'conv_out_channels'] = 64 

In [4]:
#if name == "martin":
#    df = df.iloc[-3:11,:]
#elif name == "pratt":
#    df = df.iloc[0:3,:]
#elif name == "charlotte":
#    df = df.iloc[3:-3,:]

In [5]:
df

,layers,activations,conv_out_channels,kernel_size,conv_stride,maxpool,dropout,batchnorm,optimizer,learning_rate,weight_decay,batch_size,num_epochs,net,train_loss,train_accs,valid_train_loss,valid_train_accs,valid_loss,valid_accs
1,1,"[<function relu at 0x000001A4246F4EA0>, <funct...",32,5,1,1,0.0,True,Adam,0.001,0.01,128,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,"[<function relu at 0x000001A4246F4EA0>, <funct...",32,5,1,1,0.0,True,Adam,0.005,0.01,128,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,"[<function relu at 0x000001A4246F4EA0>, <funct...",32,5,1,1,0.0,True,Adam,0.010,0.01,128,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,[<built-in method tanh of type object at 0x000...,32,5,1,1,0.0,True,Adam,0.001,0.01,128,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,[<built-in method tanh of type object at 0x000...,32,5,1,1,0.0,True,Adam,0.005,0.01,128,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,[<built-in method tanh of type object at 0x000...,32,5,1,1,0.0,True,Adam,0.010,0.01,128,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in list(df.index):#range(len(df)):
    print('Model: ', i)
    layers = df.loc[i,'layers']
    activations = df.loc[i,'activations']
    conv_out_channels = df.loc[i,'conv_out_channels']
    kernel_size = df.loc[i,'kernel_size']
    conv_stride = df.loc[i,'conv_stride']
    maxpool = int(df.loc[i,'maxpool'])
    dropout = df.loc[i,'dropout']
    batchnorm = df.loc[i,'batchnorm']
    optimizer = df.loc[i,'optimizer']
    learning_rate = df.loc[i,'learning_rate']
    weight_decay = df.loc[i,'weight_decay']
    batch_size = int(df.loc[i,'batch_size'])
    num_epochs = df.loc[i,'num_epochs']
    
    net = network_tuning.tune_architecture(layers, activations, IMAGE_SHAPE, conv_out_channels, kernel_size,conv_stride, maxpool, dropout, batchnorm)
    net_trained, df.at[i,'train_loss'], df.at[i,'train_accs'], df.at[i,'valid_train_loss'], df.at[i,'valid_train_accs'] = network_tuning.tune_train(net, data_train, data_valid, optimizer, learning_rate, weight_decay, batch_size, num_epochs)
    df.at[i,'valid_loss'], df.at[i,'valid_accs'] = network_tuning.tune_valid(net, data_valid, batch_size)
    df.at[i,'net'] = run+name+'_model'+str(i)
    df.to_pickle(run+name+'_df.pkl')
    torch.save(net_trained.state_dict(), run+name+'_model'+str(i))

Model:  1
No GPU available.
Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4


In [ ]:
#plt.plot(pd.Series(df.loc[4,'train_accs']).rolling(window=600).mean())

In [ ]:
#pd.Series(df.loc[1,'train_accs']).rolling(window=600).mean()

In [8]:
1*1200/60

20.0

In [9]:
df

,layers,activations,conv_out_channels,kernel_size,conv_stride,maxpool,dropout,batchnorm,optimizer,learning_rate,weight_decay,batch_size,num_epochs,net,train_loss,train_accs,valid_loss,valid_accs
4,2,"[<function relu at 0x000001C29EBF4EA0>, <funct...",64,5,1,1,0.0,False,Adam,0.001,0.0,128,5,NaN,NaN,NaN,NaN,NaN
5,1,[<built-in method tanh of type object at 0x000...,32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_charlotte_model5,"[0.783683717250824, 6.926929950714111, 2.30520...","[0.484375, 0.515625, 0.5859375, 0.6328125, 0.6...",0.278731,0.881915
6,1,[<built-in method tanh of type object at 0x000...,128,5,1,1,0.0,False,Adam,0.001,0.0,128,5,run1_charlotte_model6,"[0.7284291982650757, 25.511159896850586, 16.01...","[0.546875, 0.484375, 0.6171875, 0.6015625, 0.6...",0.280456,0.89017
7,2,[<built-in method tanh of type object at 0x000...,32,5,1,1,0.0,False,Adam,0.001,0.0,128,5,NaN,NaN,NaN,NaN,NaN
